In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [7]:
## Defining input and target variables 
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Defining data-frames to store results
md1_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})
md2_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})
md3_results = pd.DataFrame({'accuracy': np.repeat(np.nan, 100), 'recall': np.repeat(np.nan, 100)})


for i in range(0, 100):
    print(i)
    ## Splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    #############
    ## Model 1 ##
    #############
    
    md1 = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(X_train, Y_train)
    pred1 = md1.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < 0.1, 0, 1)
    md1_results.loc[i, 'accuracy'] = accuracy_score(Y_test, pred1)
    md1_results.loc[i, 'recall'] = recall_score(Y_test, pred1)
    
    #############
    ## Model 2 ##
    #############
    
    md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)
    pred2 = md2.predict_proba(X_test)[:, 1]
    pred2 = np.where(pred2 < 0.1, 0, 1)
    md2_results.loc[i, 'accuracy'] = accuracy_score(Y_test, pred2)
    md2_results.loc[i, 'recall'] = recall_score(Y_test, pred2)
    
    #############
    ## Model 3 ##
    #############
    
    md3 = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)
    pred3 = md3.predict_proba(X_test)[:, 1]
    pred3 = np.where(pred3 < 0.1, 0, 1)
    md3_results.loc[i, 'accuracy'] = accuracy_score(Y_test, pred3)
    md3_results.loc[i, 'recall'] = recall_score(Y_test, pred3)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
md1_results.mean()

accuracy    0.453087
recall      0.855536
dtype: float64

In [11]:
md1_results.describe()

,accuracy,recall
count,100.000000,100.000000
mean,0.453087,0.855536
std,0.020950,0.037697
min,0.401639,0.723214
25%,0.442623,0.837054
50%,0.452186,0.866071
75%,0.465847,0.883929
max,0.501366,0.919643


In [9]:
md2_results.mean()

accuracy    0.291325
recall      0.932500
dtype: float64

In [12]:
md2_results.describe()

,accuracy,recall
count,100.000000,100.000000
mean,0.291325,0.932500
std,0.066788,0.041866
min,0.153005,0.821429
25%,0.256148,0.901786
50%,0.303962,0.928571
75%,0.343238,0.964286
max,0.387978,1.000000


In [10]:
md3_results.mean()

accuracy    0.153005
recall      1.000000
dtype: float64

In [13]:
md3_results.describe()

,accuracy,recall
count,1.000000e+02,100.0
mean,1.530055e-01,1.0
std,2.789540e-16,0.0
min,1.530055e-01,1.0
25%,1.530055e-01,1.0
50%,1.530055e-01,1.0
75%,1.530055e-01,1.0
max,1.530055e-01,1.0


In [14]:
## Defining input and target variables 
X = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

md3 = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)
pred3 = md3.predict_proba(X_test)[:, 1]
# pred3
# pred3 = np.where(pred3 < 0.1, 0, 1)

array([0.12146577, 0.12146577, 0.1544571 , 0.23502385, 0.1544571 ,
       0.12146577, 0.12421822, 0.22482372, 0.1544571 , 0.12146577,
       0.12146577, 0.24076551, 0.1544571 , 0.16135844, 0.16548041,
       0.1544571 , 0.12146577, 0.1544571 , 0.1544571 , 0.16578083,
       0.1723315 , 0.12146577, 0.12146577, 0.12421822, 0.12146577,
       0.12146577, 0.12146577, 0.16578083, 0.1544571 , 0.12146577,
       0.16578083, 0.2382956 , 0.18850212, 0.16976357, 0.12146577,
       0.1544571 , 0.15842261, 0.12146577, 0.17846648, 0.12146577,
       0.12146577, 0.12146577, 0.1544571 , 0.12146577, 0.1544571 ,
       0.12146577, 0.23009576, 0.12421822, 0.1544571 , 0.1544571 ,
       0.1544571 , 0.12146577, 0.22125972, 0.12421822, 0.12146577,
       0.19938282, 0.16578083, 0.1544571 , 0.12146577, 0.16578083,
       0.18282192, 0.12421822, 0.18282192, 0.12146577, 0.16578083,
       0.12146577, 0.12146577, 0.12146577, 0.1544571 , 0.12146577,
       0.12146577, 0.1544571 , 0.12421822, 0.16135844, 0.15445